In [3]:
import torch
import dgl
from torch.optim import SparseAdam
from torch.utils.data import DataLoader
from dgl.nn.pytorch import MetaPath2Vec

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [11]:
#!pip install dgl
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.8 MB/s eta 0:00:00


In [44]:
from torch_geometric.datasets import AMiner
path = "DSAIL/AMiner"
dataset = AMiner(path, transform=None, pre_transform=None)
graph = dataset[0]

DSAIL/  sample_data/


In [49]:
graph

HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)

In [54]:
graph['paper']

{'num_nodes': 3194405}

In [64]:
graph.edge_items()

[(('paper', 'written_by', 'author'),
  {'edge_index': tensor([[      0,       1,       2,  ..., 3194404, 3194404, 3194404],
          [      0,       1,       2,  ...,    4393,   21681,  317436]])}),
 (('author', 'writes', 'paper'),
  {'edge_index': tensor([[      0,       1,       2,  ...,    4393,   21681,  317436],
          [      0,       1,       2,  ..., 3194404, 3194404, 3194404]])}),
 (('paper', 'published_in', 'venue'),
  {'edge_index': tensor([[      0,       1,       2,  ..., 3194402, 3194403, 3194404],
          [   2190,    2190,    2190,  ...,    3148,    3148,    3148]])}),
 (('venue', 'publishes', 'paper'),
  {'edge_index': tensor([[   2190,    2190,    2190,  ...,    3148,    3148,    3148],
          [      0,       1,       2,  ..., 3194402, 3194403, 3194404]])})]

In [69]:
graph.node_items()
node_counts = {key: value['num_nodes'] for key, value in graph.node_items()}
print(node_counts)
sum(node_counts.values())

{'author': 1693531, 'venue': 3883, 'paper': 3194405}


4891819

In [29]:
graph["venue"]

{'y': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]), 'y_index': tensor([1741, 2245,  111,  837, 2588, 2116, 2696, 3648, 3784,  313, 3414,  598,
        2995, 2716, 1423,  783, 1902, 3132, 1753, 2748, 2660, 3182,  775, 3339,
        1601, 3589,  156, 1145,  692, 3048,  925, 1587,  820, 1374, 3719,  819,
         492, 3830, 2777, 3001, 3693,  517, 1808, 2353, 3499, 1763, 2372, 1030,
         721, 2680, 3355, 1217, 3400, 1271, 1970, 1127,  407,  353, 1471, 1095,
         477, 3701,   65, 1009, 1899, 1442, 2073, 3143, 2466,  289, 1996, 1070,
        3871, 3695,  281, 3633,   50, 2642, 1925, 1

In [35]:
graph.node_types

['author', 'venue', 'paper']

In [21]:
metapaths = [
    ['author', 'paper', 'author'],
    ['author', 'paper','venue','paper','author']
]


In [40]:
nodes =graph.node_items()
print(nodes[2])

('paper', {'num_nodes': 3194405})


In [ ]:
print()

In [ ]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
import random
import gensim
import torch.nn.functional as F

class metapath2vecpp(nn.Module):
    def __init__(self, graph = graph, walk_per_node = 1000, lr = 1e-3, k = 5, M = 10, walk_length = 20, num_negative_samples = 5, embedding_dim = 128, metapath = metapaths[0]): # APA
      super().__init__()
      self.walk_per_node = walk_per_node
      self.walk_length = walk_length
      self.num_negative_samples = num_negative_samples
      self.embedding_dim = embedding_dim
      self.metapath = metapath
      self.k = k # window
      self.M = M # negative sample size

      self.lr = lr
      self.optimizer = optim.SGD(self.parameters(), lr=self.lr)

      node_counts = {key: value['num_nodes'] for key, value in graph.node_items()}
      self.num_node = sum(node_counts.values())

      # 학습의 대상
      self.X = nn.Embedding(self.num_node, self.embedding_dim)

      self.author_paper = {}
      self.paper_author = {}
      self.venue_paper = {}
      self.paper_venue = {}

      edges = graph.edge_items()
      paper_authors= edges[0][1]['edge_index'] # (2, 9323605) holy shittt
      venue_paper = edges[2][1]['edge_index']
      i = 0
      for paper in paper_authors[0]:
        author = paper_authors[1][i]
        i += 1
        if paper in self.paper_author:
          self.paper_author[paper].append(author)
        else:
          self.paper_author[paper] = list(author)
        if author in self.author_paper:
          self.author_paper[author].append(paper)
        else:
           self.author_paper[author]= list(paper)
      i = 0
      for venue in venue_paper[0]:
        paper = venue_paper[1][i]
        i += 1
        if paper in self.paper_venue:
          self.paper_venue[paper].append(venue)
        else:
          self.paper_venue[paper] = list(venue)
        if venue in self.venue_paper:
          self.venue_paper[venue].append(paper)
        else:
           self.venue_paper[venue]= list(paper)

       # AP / VP dictionary 완성

    def metapathRandomWalk(self, start_node, metapath): # APA인 경우
      ## start_node가 venue면 return
      if start_node in self.venue_paper:
        return
      walks = [start_node]
      # start가 A
      if start_node in self.author_paper:
        shift = 1
      else:
        shift = 0  # P
      curr_node = start_node

      # start node가 A면 P로 시작 P면 A로 시작
      for walk in range(self.walk_length - 1):
        node_type = self.metapath[(walk + shift) % len(self.metapath) ] # 0 번째는 A 3 번째도 A ...
        if node_type == "'author":
          authors = self.paper_author[curr_node]
          # 갈데 없으면 return
          if len(authors) == 0:
            return walks
          random_author = random.choice(authors)
          walks.append(random_author)
          curr_node = random_author
      return walks


    def heterogeneousSkipGram(self, k, MP):
      # deepwalk에서는 이래도 되는데 여기서는 negative sampling을 추가해줘야 함!
      # w2v = gensim.models.word2vec.Word2Vec(MP, size=self.embedding, window= k, workers=3)
      total_loss = 0

      for i in range(self.walk_length):
        v = MP[i]
        Xv = self.X(v)
        j = (i - k) if (i - k) > 0 else 0
        # k 개 skip gram
        for jj in range(j):
          ct = MP[jj]

          Xct = self.X(ct)
          # postivie 값은 maximize가 목표
          total_loss -= F.logsigmoid(torch.mul(Xv, Xct))
        # negative sampling을 같은 type안에서만 M개
        if v in self.author_paper:
          # type이 author
          random_list = list(self.author_paper.keys())
          for m in range(self.M):
            random_choice = random.choice(random_list)
            if v == random_choice:
              random_choice = random.choice(random_list)
            total_loss += F.logsigmoid(torch.mul(Xv, self.X(random_choice)))
        else:
          # type이 paper
          random_list = list(self.paper_author.keys())
          for m in range(self.M):
            random_choice = random.choice(random_list)
            if v == random_choice:
              random_choice = random.choice(random_list)
            total_loss += F.logsigmoid(torch.mul(Xv, self.X(random_choice)))

      return total_loss

    def fit(self):
      train_loss = []

      for epoch in range(self.epochs):
        total_loss = 0
        for walk in range(self.walk_per_node):
          for node in range(self.num_node):
            mp = self.metapathRandomWalk(node, self.metapath)
            loss = self.heterogeneousSkipGram(self.k, mp)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()

        train_loss.append(total_loss)
      return train_loss



In [ ]:
model = metapath2vecpp(g)

In [6]:
g = dgl.heterograph({
    ('author', 'ap', 'paper'): ([0, 0, 1, 1, 2], [0, 1, 1, 2, 2]),
    ('paper', 'pa', 'author'): ([0, 1, 1, 2, 2], [0, 0, 1, 1, 2]),
    ('paper', 'pv', 'venue'): ([0, 1, 2], [0, 0, 1]),
    ('venue', 'vp', 'paper'): ([0, 0, 1], [0, 1, 2])
})

## 의문인점

https://ericdongyx.github.io/metapath2vec/m2v.html

저자의 코드는 왜 AVA로 metapath를 설정했을까,,,

논문에서는 APA / APVPA가 제일 효과적이라고 했는뎅,,,,,,,,,,,



self.X = nn.Embedding(self.num_node, self.embedding_dim)
이거 num_node 에다가 +1해서 missing link??? 한다는 말도 있음,, 아돈노